# Computer Vision Course (2024 Spring) Homework 1

---

## Problem 4. Estimation of 2D homography
In this problem, we discuss the projective transformation between two planes. Given three sets of 2D correspondences, which are with different level of noise. Please use the DLT (Direct Linear Transformation) algorithm to estimate the 2D homography for each set of correspondences. Then calculate the reprojection error of each dataset and discuss the results. Can you come up with a method to reduce the reprojection error? Please implement your method and explain why it works. (25 points)

- Environment:
  - We suggest to use `python 3.8` with `numpy`, `opencv-python` packages to solve this problem.

- Input data: 
  - Files: `data/cv_hw_1_4_correspondences0.txt`, `data/cv_hw_1_4_correspondences1.txt`, `data/cv_hw_1_4_correspondences2.txt`
  - Format: In each row, (x1, y1, x2, y2) are the corresponding points between two planes.

- Algorithm steps:
  - Implement the basic DLT algorithm. (10 points)
  - Calculate the reprojection error. (5 points)
  - Implement a better method and explain it. (10 points)

- Hint:
  - Your answer will be graded based on both the performance of the algorithm (in Code blocks) and the completeness of the discussion (in Markdown blocks or the PDF report).
  - You can use `cv2.findHomography()` function. At the same time, please explain the chosen algorithm and the parameters in your report.
  - You are also encouraged to implement the Gold Standard algorithm without using `cv2.findHomography()`, which may earn more points.

---

### Step 1: implement the basic DLT algorihtm

* To begin with, import some necessary codebase and read the data files.

In [15]:
import numpy as np
import scipy.linalg as sl
from scipy.optimize import leastsq
import cv2

# read the data files
data0 = np.loadtxt('data/cv_hw_1_4_correspondences0.txt')
data1 = np.loadtxt('data/cv_hw_1_4_correspondences1.txt')
data2 = np.loadtxt('data/cv_hw_1_4_correspondences2.txt')
datas = [data0, data1, data2]

* To determining H given a set of four 2D to 2D point correspondences, $\mathbf{x}_i \leftrightarrow \mathbf{x}'_i$. The transform is given by $\mathbf{x}'_i = H\mathbf{x}_i$. Note that this is an equation involving homogeneous vectors; thus the 3-vectors $\mathbf{x}'_i$ and $H\mathbf{x}_i$ are not equal, they have the same direction but may differ in magnitude by a non-zero scale factor. The equation may be expressed in terms of the vector cross product as $\mathbf{x}'_i \times H\mathbf{x}_i = \mathbf{0}$.

* If the $j$-th row of the matrix $H$ is denoted by $h^{jT}$, writing $\mathbf{x}'_i = (x'_i,y'_i,w'_i)$, we can transform the equation $\mathbf{x}'_i \times H\mathbf{x}_i = \mathbf{0}$ in the form:
$$
\left[
    \begin{matrix}
        \mathbf{0}^T & -\omega'_i\mathbf{x}_i^T & y'_i\mathbf{x}_i^T \\
        \omega'_i\mathbf{x}_i^T & \mathbf{0}^T & x'_i\mathbf{x}_i^T \\
        y'_i\mathbf{x}_i^T & x'_i\mathbf{x}_i^T & \mathbf{0}^T
    \end{matrix}
\right]
\left(
    \begin{matrix}
        \mathbf{h}^1 \\
        \mathbf{h}^2 \\
        \mathbf{h}^3
    \end{matrix}
\right) = \mathbf{0} \tag{1}
$$
These equations have the form $A_i\mathbf{h} = \mathbf{0}$, where $A_i$ is a 3×9 matrix, and $\mathbf{h}$ is a 9-vector made up of the entries of the matrix $H$.

* Although there are three equations in (1), only two of them are linearly independent (since the third row is obtained, up to scale, from the sum of $x'_i$ timesthe ﬁrst row and $y'_i$ times the second). It is usual to omit the third equation in solving for $H$. Then the set of equations becomes
$$
\left[
    \begin{matrix}
        \mathbf{0}^T & -\omega'_i\mathbf{x}_i^T & y'_i\mathbf{x}_i^T \\
        \omega'_i\mathbf{x}_i^T & \mathbf{0}^T & x'_i\mathbf{x}_i^T
    \end{matrix}
\right]
\left(
    \begin{matrix}
        \mathbf{h}^1 \\
        \mathbf{h}^2 \\
        \mathbf{h}^3
    \end{matrix}
\right) = \mathbf{0} \tag{2}
$$

* When we have 4 non-collinear points, matrix $A$ has rank 8, and thus has a 1-dimensional null-space which provides a solution for $\mathbf{h}$. Such a solution $\mathbf{h}$ can only be determined up to a non-zero scale factor. A scale may be arbitrarily chosen for $\mathbf{h}$ by a requirement on its norm such as $\Vert \mathbf{h}\Vert = 1$.

In [16]:
print("Using the first four point correspondences to determine 2D homography.")
for i in range(3):
    data = datas[i]
    # create matrix A
    A = []
    for points in data[:4]:
        # convert to homogeneous coordinates
        x_i = np.append(points[:2], 1)
        # omega is 1
        A_i1 = np.hstack((np.zeros((3)), -1*x_i, points[3]*x_i))
        A_i2 = np.hstack((x_i, np.zeros((3)), -1*points[2]*x_i))
        A.append(A_i1)
        A.append(A_i2)
    A = np.array(A)

    # solve the null space of A
    h = sl.null_space(A)
    # normalization and reshape
    h = h / np.linalg.norm(h)
    H1 = h.reshape(3,3)
    print('The 2D homography for correspondences%d is:\n'%i, H1)

Using the first four point correspondences to determine 2D homography.
The 2D homography for correspondences0 is:
 [[7.62787309e-03 2.61859029e-03 4.46858412e-01]
 [1.20964572e-02 3.68300703e-02 8.93716824e-01]
 [4.46858412e-05 8.93716824e-05 4.46858412e-03]]
The 2D homography for correspondences1 is:
 [[ 4.56857775e-03 -5.82448367e-04 -1.38426764e-01]
 [ 1.70534381e-02  3.84839715e-03 -9.90203850e-01]
 [ 5.46245650e-05  6.79514125e-06 -2.74271810e-03]]
The 2D homography for correspondences2 is:
 [[-5.02509745e-03  3.19881489e-03  1.03962854e-01]
 [-2.00264688e-02  6.06667239e-03  9.94339046e-01]
 [-6.36336014e-05  2.25461079e-05  2.86967357e-03]]


* If more than four point correspondences $\mathbf{x}_i \leftrightarrow \mathbf{x}'_i$ are given, the solution is the (unit) eigenvector of $A^TA$ with least eigenvalue.

In [17]:
print("Using all point correspondences to determine 2D homography.")
for i in range(3):
    data = datas[i]
    # create matrix A
    A = []
    for points in data:
        # convert to homogeneous coordinates
        x_i = np.append(points[:2], 1)
        # omega is 1
        A_i1 = np.hstack((np.zeros((3)), -1*x_i, points[3]*x_i))
        A_i2 = np.hstack((x_i, np.zeros((3)), -1*points[2]*x_i))
        A.append(A_i1)
        A.append(A_i2)
    A = np.array(A)

    # solve the eigenvector of A^TA with least eigenvalue
    eigenvalues, eigenvectors = np.linalg.eig(A.T@A)
    min_index = np.argmin(eigenvalues)
    h = eigenvectors[:, min_index]
    # normalization and reshape
    h = h / np.linalg.norm(h)
    H2 = h.reshape(3,3)
    print('The 2D homography for correspondences%d is:\n'%i, H2)

Using all point correspondences to determine 2D homography.
The 2D homography for correspondences0 is:
 [[7.62787674e-03 2.61859206e-03 4.46858481e-01]
 [1.20964649e-02 3.68300870e-02 8.93716789e-01]
 [4.46858678e-05 8.93717253e-05 4.46858461e-03]]
The 2D homography for correspondences1 is:
 [[ 2.05378679e-03 -1.69807562e-04  3.37153940e-01]
 [ 3.95500628e-04  1.07989222e-02  9.41378016e-01]
 [ 4.06368338e-06  2.25044395e-05  3.69196454e-03]]
The 2D homography for correspondences2 is:
 [[-1.61035432e-03 -1.79041900e-03  2.74992153e-01]
 [-7.24228934e-03 -4.87886896e-03  9.61398595e-01]
 [-2.25356725e-05 -1.71597897e-05  3.16275147e-03]]


* The `cv2.findHomography` function is a part of the OpenCV library, used to find a perspective transformation between two planes, i.e., a homography matrix. The function prototype generally looks like this:

`cv2.findHomography(srcPoints, dstPoints, method=0, ransacReprojThreshold=5.0, mask=None, maxIters=2000, confidence=0.995)
`

* Just introduce the meanings of the first three parameters:
    * `srcPoints`: The coordinate array of points in the source image. These are the points to undergo the perspective transformation. It should be a numpy array of size Nx2, where N is the number of points.
    * `dstPoints`: The coordinate array of points in the destination image, corresponding to the points in `srcPoints`. It should also be a numpy array of size Nx2.
    * `method`: The method to compute the homography matrix. The default value is 0 (i.e., using all points), and common methods include:
        * `0`: A regular method using all points (if no other parameters are specified).
        * `cv2.RANSAC`: Robust estimation using the RANSAC algorithm. Suitable for cases with outliers.
        * `cv2.LMEDS`: Least Median of Squares algorithm. Another robust method but not as precise as RANSAC.
        * `cv2.RHO`: Another robust estimation method aimed to provide faster computation than RANSAC while maintaining similar robustness.
* Return Values:
    * `H`: The homography matrix, i.e., the perspective transformation matrix.
    * `status`: The mask parameter mentioned above, indicating which points are inliers.


In [18]:
print("Using cv2.findHomography() to determine 2D homography.")
for i in range(3):
    data = datas[i]
    # create matrix A
    H3, status = cv2.findHomography(data[:,:2], data[:,2:], 0)
    H3 = H3 / np.linalg.norm(H3)
    print('The 2D homography for correspondences%d is:\n'%i, H3)

Using cv2.findHomography() to determine 2D homography.
The 2D homography for correspondences0 is:
 [[7.62790845e-03 2.61861814e-03 4.46858786e-01]
 [1.20965142e-02 3.68302741e-02 8.93716628e-01]
 [4.46860570e-05 8.93722337e-05 4.46858803e-03]]
The 2D homography for correspondences1 is:
 [[8.26610922e-03 2.94643336e-03 4.37899042e-01]
 [1.27836432e-02 3.92424051e-02 8.98021963e-01]
 [4.73726842e-05 9.58338127e-05 4.53582307e-03]]
The 2D homography for correspondences2 is:
 [[ 6.23477130e-02  3.26265084e-02  9.04988536e-02]
 [ 1.06924322e-01  2.55122281e-01 -9.54086048e-01]
 [ 3.31777503e-04  6.59750184e-04  7.57031081e-03]]


### Step 2: calculate the reprojection error

* We compute the error for 2D homography $H$ using all point correspondences to determine.

* Assuming the coordinates of points in the first plane are accurate (with no noise), then the reprojection error degenerates to error in one image. We compute this error in the following code block:

In [19]:
# error in one image
for i in range(3):
    data = datas[i]
    # create matrix A
    one_colunm = np.ones((len(data), 1))
    ini_x = np.hstack((data[:,:2], one_colunm))
    trans_x = H2@ini_x.T
    trans_x /= trans_x[2] # divided by scale para
    error = np.sum((trans_x.T[:,:2]- data[:,2:])**2)
    print('The error in one image for correspondences%d is:\n'%i, error)

The error in one image for correspondences0 is:
 2950.2873431525127
The error in one image for correspondences1 is:
 11906.52029527249
The error in one image for correspondences2 is:
 4504.057120426997


* If all corresponding points between two planes have noise, then we may compute the reprojection error by minimizing the error function $\sum_i d(\mathbf{x}_i, \hat{\mathbf{x}}_i)^2 + d(\mathbf{x}'_i, \hat{\mathbf{x}}'_i)^2$ using Levenberg-Marquardt algorithm over $n$ 2D points $\hat{\mathbf{x}}_i$.

In [20]:
# using scipy.optimize.leastsq to minimize using L-M algorithm
def residuals1(ini_x_hat, x, H):
    '''
    ini_x_hat: the perfectly matched corresponding point in first plane. Since the leastsq setting, it's 2N*1 matrix
    x: corresponding points with noise. N*4 matrix
    H: 2D homography matrix. 3*3 matrix
    '''
    # reshape ini_x_hat to N*2
    ini_x_hat = ini_x_hat.reshape(len(ini_x_hat)//2, 2)
    one_colunm = np.ones((len(ini_x_hat), 1))
    ini_x_hat = np.hstack((ini_x_hat, one_colunm))
    trans_x_hat = H@ini_x_hat.T
    trans_x_hat /= trans_x_hat[2] # divided by scale para
    trans_x_hat = trans_x_hat.T
    x_hat = np.hstack((ini_x_hat[:,:2], trans_x_hat[:,:2]))
    # mind that leastsq func needs residuals func return a 1D array
    return (x - x_hat).flatten()

for i in range(3):
    data = datas[i]
    # set the points in 1st plane as original x
    x0 = data[:,:2]
    result = leastsq(residuals1, x0.flatten(), args=(data, H2))
    reprojection_error = np.sum(residuals1(result[0], data, H2)**2)
    print('The reprojection error for correspondences%d is:\n'%i, reprojection_error)

The reprojection error for correspondences0 is:
 720.0747132765634
The reprojection error for correspondences1 is:
 690.6220555902507
The reprojection error for correspondences2 is:
 2035.3641339908454


### Step3: implement a better method and explain it

* There will usually be one cost function which is optimal in the sense that the $H$ that minimizes it gives the best possible estimate of the transformation under certain assumptions. The computational algorithm that enables this cost function to be minimized is called the "Gold Standard" algorithm. The results of other algorithms are assessed by how well they compare to this Gold Standard. The algorithm is shown as the following:

![Gold standard error](./Gold_standard_error.png)

In [21]:
# find the Gold Standard Error
def residuals2(ini_x_hat_and_H, x):
    '''
    ini_x_hat_and_H: variables, the perfectly matched corresponding point in first plane and 2D homograaphy H. (2N+9)*1 matrix
    x: corresponding points with noise. N*4 matrix
    '''
    # reshape ini_x_hat to N*2
    ini_x_hat = ini_x_hat_and_H[:-9]
    H = ini_x_hat_and_H[-9:]
    ini_x_hat = ini_x_hat.reshape(len(ini_x_hat)//2, 2)
    H = H.reshape(3,3)
    one_colunm = np.ones((len(ini_x_hat), 1))
    ini_x_hat = np.hstack((ini_x_hat, one_colunm))
    trans_x_hat = H@ini_x_hat.T
    trans_x_hat /= trans_x_hat[2] # divided by scale para
    trans_x_hat = trans_x_hat.T
    x_hat = np.hstack((ini_x_hat[:,:2], trans_x_hat[:,:2]))
    # mind that leastsq func needs residuals func return a 1D array
    return (x - x_hat).flatten()

for i in range(3):
    data = datas[i]
    # set the points in 1st plane as original x
    x0 = data[:,:2]
    x0_and_H = np.hstack((x0.flatten(), H2.flatten()))
    result = leastsq(residuals2, x0_and_H, args=(data))
    reprojection_error = np.sum(residuals2(result[0], data)**2)
    print('The gold standard error for reprojection error for correspondences%d is:\n'%i, reprojection_error)
    print('And the 2D homography is:\n', x0_and_H[-9:].reshape(3,3))

The gold standard error for reprojection error for correspondences0 is:
 1.5045155184375137e-26
And the 2D homography is:
 [[-1.61035432e-03 -1.79041900e-03  2.74992153e-01]
 [-7.24228934e-03 -4.87886896e-03  9.61398595e-01]
 [-2.25356725e-05 -1.71597897e-05  3.16275147e-03]]
The gold standard error for reprojection error for correspondences1 is:
 4.761759542800305
And the 2D homography is:
 [[-1.61035432e-03 -1.79041900e-03  2.74992153e-01]
 [-7.24228934e-03 -4.87886896e-03  9.61398595e-01]
 [-2.25356725e-05 -1.71597897e-05  3.16275147e-03]]
The gold standard error for reprojection error for correspondences2 is:
 665.7623293183253
And the 2D homography is:
 [[-1.61035432e-03 -1.79041900e-03  2.74992153e-01]
 [-7.24228934e-03 -4.87886896e-03  9.61398595e-01]
 [-2.25356725e-05 -1.71597897e-05  3.16275147e-03]]
